# Scratchpad for paper revisions

In [1]:
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path

from utils.data import make_dataset
from utils.nnet import get_device

from hebbcl.logger import MetricLogger
from hebbcl.model import Nnet
from hebbcl.trainer import Optimiser, train_model
from hebbcl.parameters import parser


In [77]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]
sorted(vars(args))


['centering',
 'ctx_avg',
 'ctx_avg_alpha',
 'ctx_avg_type',
 'ctx_avg_window',
 'ctx_scaling',
 'ctx_w_init',
 'ctx_weights',
 'cuda',
 'device',
 'gating',
 'hebb_normaliser',
 'log_interval',
 'loss_funct',
 'lrate_hebb',
 'lrate_sgd',
 'n_episodes',
 'n_features',
 'n_hidden',
 'n_out',
 'n_runs',
 'perform_hebb',
 'perform_sgd',
 'save_dir',
 'save_results',
 'training_schedule',
 'verbose',
 'weight_init']

In [105]:
# override defaults 
args.gating = "oja"
args.perform_hebb = True
args.centering = True
args.lrate_hebb = 1e-3
args.lrate_sgd = 1e-1
args.ctx_scaling = 3

In [106]:
# create dataset 
dataset = make_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)

In [107]:

# train model
train_model(args, model, optimiser, dataset, logger)

step 0, loss: task a -0.0984, task b 0.1328 | acc: task a 0.7500, task b 0.5000
... n_a: 6 n_b: 8
step 50, loss: task a -8.5885, task b -0.0563 | acc: task a 1.0000, task b 0.5000
... n_a: 12 n_b: 1
step 100, loss: task a -13.6945, task b -0.0655 | acc: task a 1.0000, task b 0.5000
... n_a: 24 n_b: 2
step 150, loss: task a -14.4477, task b -0.0068 | acc: task a 1.0000, task b 0.5000
... n_a: 30 n_b: 2
step 200, loss: task a -14.6889, task b 0.0376 | acc: task a 1.0000, task b 0.5000
... n_a: 34 n_b: 1
step 250, loss: task a -14.7949, task b -0.0229 | acc: task a 1.0000, task b 0.5000
... n_a: 35 n_b: 1
step 300, loss: task a -14.8522, task b 0.0202 | acc: task a 1.0000, task b 0.5000
... n_a: 35 n_b: 1
step 350, loss: task a -14.8889, task b 0.0526 | acc: task a 1.0000, task b 0.5000
... n_a: 36 n_b: 1
step 400, loss: task a -14.9127, task b 0.0226 | acc: task a 1.0000, task b 0.5000
... n_a: 36 n_b: 1
step 450, loss: task a -14.9298, task b 0.0340 | acc: task a 1.0000, task b 0.5000
.

## hyperparameter optimisation
I've changed the oja update to be applied to all hidden units. Now networks no longer converge.
Let's run HPO to find best-fitting lrate etc

In [15]:
import ray
ray.shutdown()

ray.init(namespace='timotest', runtime_env={'excludes': ['d:\DPHIL_02_CONTLEARN\paper\codebase\notebooks\..\.git\objects\pack\pack-5d9ef272208dd160d8cc458b72ef8f6ef1f54140.pack'],"working_dir": "../"})
from ray import tune 
from hebbcl.tuner import run_raytune

2022-06-16 16:49:05,103	INFO packaging.py:353 -- Creating a file package for local directory '../'.
2022-06-16 16:49:05,144	WARNING packaging.py:258 -- File d:\DPHIL_02_CONTLEARN\paper\codebase\notebooks\..\.git\objects\pack\pack-5d9ef272208dd160d8cc458b72ef8f6ef1f54140.pack is very large (189.49MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['d:\DPHIL_02_CONTLEARN\paper\codebase\notebooks\..\.git\objects\pack\pack-5d9ef272208dd160d8cc458b72ef8f6ef1f54140.pack']})`


RuntimeError: Package size (191.20MiB) exceeds the maximum size of 100.00MiB. You can exclude large files using the 'excludes' option to the runtime_env.

In [120]:
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.gating = "oja"
args.perform_hebb = True
args.centering = True
args.lrate_hebb = 1e-3
args.lrate_sgd = 1e-1
args.ctx_scaling = 3

In [121]:
# let's have a look at all params
dict(sorted(vars(args).items(),key=lambda k: k[0]))

{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 3,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'device': device(type='cpu'),
 'gating': 'oja',
 'hebb_normaliser': 10.0,
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.001,
 'lrate_sgd': 0.1,
 'n_episodes': 200,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}

In [123]:
config = {
    "lrate_sgd": tune.loguniform(1e-4, 5e-1),
    "lrate_hebb": tune.loguniform(1e-4, 5e-1),
    "ctx_scaling": tune.uniform(1, 10),   
    
}

results = run_raytune(config, args)

2022-06-16 16:35:13,827	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-06-16 16:35:14,210	ERROR syncer.py:111 -- Log sync requires rsync to be installed.
2022-06-16 16:35:15,285	WARNING worker.py:1257 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 670, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 636, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 639, in actor_method_executor
    return method(__ray_actor, *args, **kwar

== Status ==
Current time: 2022-06-16 16:35:15 (running for 00:00:01.29)
Memory usage on this node: 8.7/15.5 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 0/1 GPUs, 0.0/4.83 GiB heap, 0.0/2.42 GiB objects
Result logdir: C:\Users\Timo\ray_results\train_nnet_with_ray_2022-06-16_16-35-13
Number of trials: 16/100 (15 PENDING, 1 RUNNING)
+---------------------------------+----------+-------+---------------+--------------+-------------+
| Trial name                      | status   | loc   |   ctx_scaling |   lrate_hebb |   lrate_sgd |
|---------------------------------+----------+-------+---------------+--------------+-------------|
| train_nnet_with_ray_ea47b_00000 | RUNNING  |       |       3.59792 |  0.00317502  | 0.00731544  |
| train_nnet_with_ray_ea47b_00001 | PENDING  |       |       9.47599 |  0.0735329   | 0.00134198  |
| train_nnet_with_ray_ea47b_00002 | PENDING  |       |       7.53001 |  0.000352109 | 0.00523461  |
| train_nnet_with_ray_ea47b_00003 | PEND

 pid=18876) 2022-06-16 16:35:15,276	ERROR worker.py:432 -- SystemExit was raised from the worker.
 pid=18876) Traceback (most recent call last):
 pid=18876)   File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
 pid=18876)   File "python\ray\_raylet.pyx", line 670, in ray._raylet.execute_task
 pid=18876)   File "python\ray\_raylet.pyx", line 636, in ray._raylet.execute_task
 pid=18876)   File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
 pid=18876)   File "python\ray\_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
 pid=18876)   File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 639, in actor_method_executor
 pid=18876)     return method(__ray_actor, *args, **kwargs)
 pid=18876)   File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 542, in temporary_actor_method
 pid=18876)     raise RuntimeError(
 pid=18876) RuntimeError: The actor

Result for train_nnet_with_ray_ea47b_00000:
  trial_id: ea47b_00000
  


2022-06-16 16:35:18,713	ERROR trial_runner.py:927 -- Trial train_nnet_with_ray_ea47b_00002: Error processing event.
Traceback (most recent call last):
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\tune\trial_runner.py", line 893, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\tune\ray_trial_executor.py", line 707, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\worker.py", line 1735, in get
    raise value
ray.exceptions.RayActorError: The actor died unexpectedly before finishing this task.
2022-06-16 16:35:18,720	WARNING worker.py:1257 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 629, in ray._r

...

2022-06-16 16:36:21,387	WARNING worker.py:1257 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 670, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 636, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 639, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 542, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call la

Result for train_nnet_with_ray_ea47b_00097:
  trial_id: ea47b_00097
  


2022-06-16 16:36:22,472	WARNING worker.py:1257 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 670, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 636, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 639, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\Timo\anaconda3\envs\pytorch\lib\site-packages\ray\_private\function_manager.py", line 542, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call la

Result for train_nnet_with_ray_ea47b_00098:
  trial_id: ea47b_00098
  
Result for train_nnet_with_ray_ea47b_00099:
  trial_id: ea47b_00099
  
== Status ==
Current time: 2022-06-16 16:36:22 (running for 00:01:08.54)
Memory usage on this node: 8.3/15.5 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/4.83 GiB heap, 0.0/2.42 GiB objects
Result logdir: C:\Users\Timo\ray_results\train_nnet_with_ray_2022-06-16_16-35-13
Number of trials: 100/100 (100 ERROR)
+---------------------------------+----------+-------+---------------+--------------+-------------+
| Trial name                      | status   | loc   |   ctx_scaling |   lrate_hebb |   lrate_sgd |
|---------------------------------+----------+-------+---------------+--------------+-------------|
| train_nnet_with_ray_ea47b_00000 | ERROR    |       |       3.59792 |  0.00317502  | 0.00731544  |
| train_nnet_with_ray_ea47b_00001 | ERROR    |       |       9.47599 |  0.0735329   | 0.00134198  |
| train_nne

TuneError: ('Trials did not complete', [train_nnet_with_ray_ea47b_00000, train_nnet_with_ray_ea47b_00001, train_nnet_with_ray_ea47b_00002, train_nnet_with_ray_ea47b_00003, train_nnet_with_ray_ea47b_00004, train_nnet_with_ray_ea47b_00005, train_nnet_with_ray_ea47b_00006, train_nnet_with_ray_ea47b_00007, train_nnet_with_ray_ea47b_00008, train_nnet_with_ray_ea47b_00009, train_nnet_with_ray_ea47b_00010, train_nnet_with_ray_ea47b_00011, train_nnet_with_ray_ea47b_00012, train_nnet_with_ray_ea47b_00013, train_nnet_with_ray_ea47b_00014, train_nnet_with_ray_ea47b_00015, train_nnet_with_ray_ea47b_00016, train_nnet_with_ray_ea47b_00017, train_nnet_with_ray_ea47b_00018, train_nnet_with_ray_ea47b_00019, train_nnet_with_ray_ea47b_00020, train_nnet_with_ray_ea47b_00021, train_nnet_with_ray_ea47b_00022, train_nnet_with_ray_ea47b_00023, train_nnet_with_ray_ea47b_00024, train_nnet_with_ray_ea47b_00025, train_nnet_with_ray_ea47b_00026, train_nnet_with_ray_ea47b_00027, train_nnet_with_ray_ea47b_00028, tra

{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 3,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'device': device(type='cpu'),
 'gating': 'oja',
 'hebb_normaliser': 10.0,
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.001,
 'lrate_sgd': 0.1,
 'n_episodes': 200,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}